In [1]:
#for basic pre-processing
import pandas as pd
import numpy as np
import re
from re import sub

In [2]:
#for building the wordtovec model
from gensim.models.phrases import Phrases, Phraser
import multiprocessing
from gensim.models import Word2Vec
from collections import defaultdict
from time import time 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import seaborn as sns

unable to import 'smart_open.gcs', disabling that module


In [3]:
#for sentiment analysis
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score 


## Pre-process the Tweets

In [4]:
biden_url = 'Biden Tweets - Biden_Tweets.csv'
biden_df = pd.read_csv(biden_url, error_bad_lines= False).drop(['Unnamed: 1', 'Username'], axis = 1)
biden_df.head(5)

,timestamp,Retweets,Text,Mentions,Hashtags,Sentiment
0,Thu Apr 25 16:59:59 +0000 2019,0,"And, we have President Trump to thank.",NaN,NaN,0.0
1,Thu Apr 25 16:59:59 +0000 2019,0,pic.twitter.com/PxypFuZzdb,NaN,NaN,0.0
2,Thu Apr 25 16:59:58 +0000 2019,0,The “hang em high” references were less than s...,NaN,NaN,0.0
3,Thu Apr 25 16:59:57 +0000 2019,0,"Donald Trump Makes Joe Biden’s Point, Mocking ...",NaN,NaN,-1.0
4,Thu Apr 25 16:59:57 +0000 2019,0,"Factually speaking, that’s not accurate. Trump...",NaN,NaN,-1.0


In [5]:
def text_to_word_list(text):

    text = str(text)
    text = text.lower()
    
    if 'pic.twitter' in text:
        text = 'empty'

    # Clean the text
    text = sub(r"[^A-Za-z0-9^,!?.\/'+]", " ", text)
    text = sub(r"\+", " plus ", text)
    text = sub(r",", " ", text)
    text = sub(r"\.", " ", text)
    text = sub(r"!", " ", text)
    text = sub(r"\?", " ", text)
    text = sub(r"'", " ", text)
    text = sub(r":", " ", text)
    text = sub(r"\s{2,}", " ", text)
    text = sub(r'http\S+', '', text)
    text = sub(r'//twitter', '', text)
    text = sub(r'//www', '', text)
    
    text = text.split()
    text = list(filter(lambda x: len(x) > 1, text))

    return text  

In [6]:
biden_df["Cleaned_Text"] = biden_df["Text"].apply(lambda x: text_to_word_list(x))
biden_df.head(5)

,timestamp,Retweets,Text,Mentions,Hashtags,Sentiment,Cleaned_Text
0,Thu Apr 25 16:59:59 +0000 2019,0,"And, we have President Trump to thank.",NaN,NaN,0.0,"[and, we, have, president, trump, to, thank]"
1,Thu Apr 25 16:59:59 +0000 2019,0,pic.twitter.com/PxypFuZzdb,NaN,NaN,0.0,[empty]
2,Thu Apr 25 16:59:58 +0000 2019,0,The “hang em high” references were less than s...,NaN,NaN,0.0,"[the, hang, em, high, references, were, less, ..."
3,Thu Apr 25 16:59:57 +0000 2019,0,"Donald Trump Makes Joe Biden’s Point, Mocking ...",NaN,NaN,-1.0,"[donald, trump, makes, joe, biden, point, mock..."
4,Thu Apr 25 16:59:57 +0000 2019,0,"Factually speaking, that’s not accurate. Trump...",NaN,NaN,-1.0,"[factually, speaking, that, not, accurate, tru..."


In [7]:
def empty(text):
    is_empty = False
    if text == ['empty']:
        is_empty = True
    return is_empty

In [8]:
def filter_df(df):
    df["is_empty"] = df["Cleaned_Text"].apply(lambda x: empty(x))
    df_filtered = df[df["is_empty"] == False]
    final_df = df_filtered.drop(["is_empty"], axis = 1)
    return final_df

In [9]:
biden_df = filter_df(biden_df)

In [10]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [11]:
#remove stop words
biden_df['Cleaned_Text'] = biden_df['Cleaned_Text'].apply(lambda x: [item for item in x if item not in stop])
biden_df

,timestamp,Retweets,Text,Mentions,Hashtags,Sentiment,Cleaned_Text
0,Thu Apr 25 16:59:59 +0000 2019,0,"And, we have President Trump to thank.",NaN,NaN,0.0,"[president, trump, thank]"
2,Thu Apr 25 16:59:58 +0000 2019,0,The “hang em high” references were less than s...,NaN,NaN,0.0,"[hang, em, high, references, less, subtle]"
3,Thu Apr 25 16:59:57 +0000 2019,0,"Donald Trump Makes Joe Biden’s Point, Mocking ...",NaN,NaN,-1.0,"[donald, trump, makes, joe, biden, point, mock..."
4,Thu Apr 25 16:59:57 +0000 2019,0,"Factually speaking, that’s not accurate. Trump...",NaN,NaN,-1.0,"[factually, speaking, accurate, trump, accused..."
5,Thu Apr 25 16:59:57 +0000 2019,0,HAAAAAA!!!,NaN,NaN,0.0,[haaaaaa]
6,Thu Apr 25 16:59:57 +0000 2019,0,So he is asking Joe Biden if he has the intell...,NaN,NaN,-1.0,"[asking, joe, biden, intelligence, president, ..."
7,Thu Apr 25 16:59:57 +0000 2019,0,"That us why, we the people, elected Trump",NaN,NaN,1.0,"[us, people, elected, trump]"
8,Thu Apr 25 16:59:57 +0000 2019,1,Donald Trump Jr. Wins the Internet After Thump...,@gatewaypundit,NaN,1.0,"[donald, trump, jr, wins, internet, thumping, ..."
9,Thu Apr 25 16:59:55 +0000 2019,0,...Time has run out for YOU Joe!,NaN,NaN,-1.0,"[time, run, joe]"
10,Thu Apr 25 16:59:54 +0000 2019,0,who cares!,NaN,NaN,-1.0,[cares]


## Create the Word2Vec Model
_Link:_ https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

In [12]:
#Word2Vec takes in a list of lists 
sent = list(biden_df['Cleaned_Text'])
sent

[['president', 'trump', 'thank'],
 ['hang', 'em', 'high', 'references', 'less', 'subtle'],
 ['donald',
  'trump',
  'makes',
  'joe',
  'biden',
  'point',
  'mocking',
  'candidate',
  'intelligence',
  'dems',
  'sick',
  'de',
  'yahoo',
  'com/entertainment/donald',
  'trump',
  'makes',
  'joe',
  'biden',
  '130340726',
  'html',
  'tsrc',
  'fauxdal'],
 ['factually',
  'speaking',
  'accurate',
  'trump',
  'accused',
  'nearly',
  'two',
  'dozen',
  'incidents',
  'sexual',
  'misconduct',
  'including',
  'rape',
  'one',
  'accused',
  'biden',
  'anything',
  'sexual',
  'far',
  'businessinsider',
  'com/women',
  'accused',
  'trump',
  'sexual',
  'misconduct',
  'list',
  '2017',
  '12'],
 ['haaaaaa'],
 ['asking',
  'joe',
  'biden',
  'intelligence',
  'president',
  'sits',
  'white',
  'house',
  'stupid',
  'grin',
  'face',
  'trump',
  'intelligence',
  'president',
  'yet'],
 ['us', 'people', 'elected', 'trump'],
 ['donald',
  'trump',
  'jr',
  'wins',
  'intern

In [13]:
phrases = Phrases(sent, min_count=30, progress_per=10000)
phrases

In [14]:
bigram = Phraser(phrases)

In [15]:
sentences = bigram[sent]
sentences

In [16]:
#sanity check
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

47709

In [17]:
sorted(word_freq, key=word_freq.get, reverse=True)[:20]

['joe_biden',
 'joebiden',
 'trump',
 'biden',
 'joe',
 'like',
 'president',
 'people',
 'get',
 'would',
 'one',
 'know',
 'think',
 'vote',
 'obama',
 'us',
 'need',
 'want',
 'going',
 'ukraine']

In [18]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [20]:
w2v_model = Word2Vec(min_count=3,
                     window=4,
                     size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

/Users/wangjiayun/anaconda3/lib/python3.7/site-packages/gensim/models/base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [21]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.04 mins


In [22]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

KeyboardInterrupt: 

In [ ]:
#makes the model memory efficient
w2v_model.init_sims(replace=True)

In [ ]:
#exploring the model
w2v_model.wv.most_similar(positive=["biden"]) #checks most similar words related to biden

In [ ]:
w2v_model.wv.most_similar(positive=["biden"])

In [ ]:
def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=21).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))

In [ ]:
tsnescatterplot(w2v_model, 'biden', [i[0] for i in w2v_model.wv.most_similar(negative=["biden"])])

In [ ]:
w2v_model.wv.most_similar(positive=["biden"])

## Sentiment Analysis

In [233]:
word_vectors = w2v_model.wv
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors)
positive_cluster_center = model.cluster_centers_[0]
negative_cluster_center = model.cluster_centers_[1]

In [183]:
#model.cluster_centers_

In [184]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

[('rubin', 0.9999436736106873),
 ('contrived', 0.9999411106109619),
 ('imprisonment', 0.9999401569366455),
 ('censorship', 0.999940037727356),
 ('duplicate', 0.9999382495880127),
 ('tha', 0.9999382495880127),
 ('teddy', 0.9999380707740784),
 ('flush', 0.999937891960144),
 ('protester', 0.9999376535415649),
 ('dressed', 0.999936044216156)]

In [185]:
words = pd.DataFrame(word_vectors.vocab.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors.wv[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])
words['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

/Users/siddhipatel/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [186]:
words

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
0,president,"[0.09005512, 0.058325462, 0.019639576, -0.0412...",0,1,4.539642,4.539642
1,trump,"[0.07960435, 0.061248135, 0.009246659, -0.0521...",0,1,4.040623,4.040623
2,thank,"[0.07881143, 0.039000057, 0.011824912, -0.0154...",0,1,12.184477,12.184477
3,hang,"[0.08196911, 0.04198753, 0.0063870465, -0.0214...",0,1,25.514709,25.514709
4,em,"[0.08443287, 0.032673717, 0.00027479776, -0.02...",0,1,13.461939,13.461939
...,...,...,...,...,...,...
14195,wisconsinprimary,"[0.08202684, 0.03854101, 0.0042507066, -0.0208...",0,1,56.541540,56.541540
14196,trumpisaloser,"[0.08076347, 0.037439734, 0.0059369355, -0.019...",0,1,34.164310,34.164310
14197,caa,"[0.08543836, 0.03832225, 0.0053931125, -0.0192...",0,1,27.470180,27.470180
14198,americaworkstogether,"[0.08238889, 0.03775217, 0.0037380834, -0.0209...",0,1,58.145002,58.145002


In [337]:
biden_df['Cleaned_Text'] = biden_df['Cleaned_Text'] .apply(lambda x: ' '.join(word for word in x))
biden_df

,timestamp,Retweets,Text,Mentions,Hashtags,Sentiment,Cleaned_Text
0,Thu Apr 25 16:59:59 +0000 2019,0,"And, we have President Trump to thank.",NaN,NaN,0.0,president trump thank
2,Thu Apr 25 16:59:58 +0000 2019,0,The “hang em high” references were less than s...,NaN,NaN,0.0,hang em high references less subtle
3,Thu Apr 25 16:59:57 +0000 2019,0,"Donald Trump Makes Joe Biden’s Point, Mocking ...",NaN,NaN,-1.0,donald trump makes joe biden point mocking can...
4,Thu Apr 25 16:59:57 +0000 2019,0,"Factually speaking, that’s not accurate. Trump...",NaN,NaN,-1.0,factually speaking accurate trump accused near...
5,Thu Apr 25 16:59:57 +0000 2019,0,HAAAAAA!!!,NaN,NaN,0.0,haaaaaa
...,...,...,...,...,...,...,...
52774,Sat Apr 11 11:49:39 +0000 2020,0,#NeverBidenNeverTrump,NaN,#NeverBidenNeverTrump,NaN,neverbidennevertrump
52776,Sat Apr 11 11:49:37 +0000 2020,0,Yes he have to for some whom haven’t hear it b...,NaN,NaN,NaN,yes hear wise reasonable know news outlets rep...
52777,Sat Apr 11 11:49:36 +0000 2020,0,"No, no... this way Joe, this way........... th...",NaN,NaN,NaN,way joe way kitchen
52779,Sat Apr 11 11:49:36 +0000 2020,0,LA voter- it literally doesn’t matter. You can...,NaN,NaN,NaN,la voter literally matter stay home rest state...


In [338]:
biden_df = biden_df[biden_df['Sentiment'] != np.nan]
biden_df = biden_df[biden_df['Sentiment'] == -1]

In [339]:
row_and_indexes = pd.DataFrame(data = biden_df[['Cleaned_Text', 'Sentiment']].reset_index(drop=True))
row_and_indexes

,Cleaned_Text,Sentiment
0,donald trump makes joe biden point mocking can...,-1.0
1,factually speaking accurate trump accused near...,-1.0
2,asking joe biden intelligence president sits w...,-1.0
3,time run joe,-1.0
4,cares,-1.0
...,...,...
650,brings child biden event encourages get ask qu...,-1.0
651,trump brazenly lies sticking joe biden unhinge...,-1.0
652,joebiden respond idiot everyone knows disgrace...,-1.0
653,joe specialty plagiarize everything get hans,-1.0


In [340]:
tfidf = TfidfVectorizer(tokenizer=lambda y: y.split(), norm=None)
tfidf.fit(biden_df['Cleaned_Text'])
features = pd.Series(tfidf.get_feature_names())
transformed = tfidf.transform(biden_df['Cleaned_Text'])
features

/Users/siddhipatel/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


0                //aflcio
1                //amazon
2                 //apple
3                   //bit
4       //canadafreepress
              ...        
3331                zarif
3332                 zero
3333                zilch
3334              zionist
3335          zurhd5bu1o8
Length: 3336, dtype: object

In [341]:
def create_tfidf_dictionary(x, transformed_file, features):
    '''
    create dictionary for each input sentence x, where each word has assigned its tfidf score
    
    inspired  by function from this wonderful article: 
    https://medium.com/analytics-vidhya/automated-keyword-extraction-from-articles-using-nlp-bfd864f41b34
    
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    vector_coo = transformed_file[x.name].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, transformed_file, features):
    '''
    replacing each word with it's calculated tfidf dictionary with scores of each word
    x - row of dataframe, containing sentences, and their indexes,
    transformed_file - all sentences transformed with TfidfVectorizer
    features - names of all words in corpus used in TfidfVectorizer
    '''
    dictionary = create_tfidf_dictionary(x, transformed_file, features)   
    return list(map(lambda y:dictionary[f'{y}'], x.Cleaned_Text.split()))
    
replaced_tfidf_scores = row_and_indexes.apply(lambda x: replace_tfidf_words(x, transformed, features), axis=1)

In [342]:
replaced_tfidf_scores

0      [4.652947444887873, 5.380740486694696, 11.0805...
1      [6.793013608384144, 6.099866427824199, 6.38754...
2      [5.876722876509988, 1.6426163719127294, 1.7431...
3      [4.541721809777648, 5.088265516145719, 1.64261...
4                                    [5.876722876509988]
                             ...                        
650    [13.586027216768288, 5.2889362116078695, 5.229...
651    [2.690370243347348, 13.586027216768288, 10.813...
652    [2.6739764335716716, 6.3875485002759795, 5.288...
653    [1.6426163719127294, 6.793013608384144, 6.7930...
654    [6.793013608384144, 5.540250639888776, 6.79301...
Length: 655, dtype: object

In [343]:
sentiment_map = words
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [344]:
def replace_sentiment_words(word, sentiment_dict):
    '''
    replacing each word with its associated sentiment score from sentiment dict
    '''
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

replaced_closeness_scores = row_and_indexes['Cleaned_Text'].apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))
replaced_closeness_scores



0      [8.233043627855018, 4.040623316285427, 10.2262...
1      [49.788483017213, 17.43111408195668, 33.119377...
2      [15.266329608207329, 3.733417775765308, 2.3957...
3      [7.089307034328262, 10.378012291354713, 3.7334...
4                                   [16.612738124542933]
                             ...                        
650    [15.17437103017262, 11.968887133700807, 2.3957...
651    [4.040623316285427, 0, 13.395935541856458, 21....
652    [-3.5097897736203434, 29.541219571205847, 17.7...
653    [3.733417775765308, 36.2548458414366, 54.19152...
654    [0, 7.187027571263889, 0, 5.511086485339778, 3...
Name: Cleaned_Text, Length: 655, dtype: object

In [345]:
replacement_df = pd.DataFrame(data=[replaced_closeness_scores, replaced_tfidf_scores, row_and_indexes['Cleaned_Text'], row_and_indexes['Sentiment']]).T
replacement_df.columns = ['sentiment_coeff', 'tfidf_scores', 'sentence', 'sentiment']
replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')
replacement_df['sentiment'] = [1 if i==1 else 0 for i in replacement_df.sentiment]
replacement_df


,sentiment_coeff,tfidf_scores,sentence,sentiment,sentiment_rate,prediction
0,"[8.233043627855018, 4.040623316285427, 10.2262...","[4.652947444887873, 5.380740486694696, 11.0805...",donald trump makes joe biden point mocking can...,0,722.612889,1
1,"[49.788483017213, 17.43111408195668, 33.119377...","[6.793013608384144, 6.099866427824199, 6.38754...",factually speaking accurate trump accused near...,0,3986.813332,1
2,"[15.266329608207329, 3.733417775765308, 2.3957...","[5.876722876509988, 1.6426163719127294, 1.7431...",asking joe biden intelligence president sits w...,0,1373.599758,1
3,"[7.089307034328262, 10.378012291354713, 3.7334...","[4.541721809777648, 5.088265516145719, 1.64261...",time run joe,0,91.136316,1
4,[16.612738124542933],[5.876722876509988],cares,0,97.628458,1
...,...,...,...,...,...,...
650,"[15.17437103017262, 11.968887133700807, 2.3957...","[13.586027216768288, 5.2889362116078695, 5.229...",brings child biden event encourages get ask qu...,0,1529.326329,1
651,"[4.040623316285427, 0, 13.395935541856458, 21....","[2.690370243347348, 13.586027216768288, 10.813...",trump brazenly lies sticking joe biden unhinge...,0,2385.718160,1
652,"[-3.5097897736203434, 29.541219571205847, 17.7...","[2.6739764335716716, 6.3875485002759795, 5.288...",joebiden respond idiot everyone knows disgrace...,0,2898.169663,1
653,"[3.733417775765308, 36.2548458414366, 54.19152...","[1.6426163719127294, 6.793013608384144, 6.7930...",joe specialty plagiarize everything get hans,0,680.538856,1


In [346]:
predicted_classes = replacement_df.prediction
y_test = replacement_df.sentiment

conf_matrix = pd.DataFrame(confusion_matrix(replacement_df.sentiment, replacement_df.prediction))
print('Confusion Matrix')
display(conf_matrix)

test_scores = accuracy_score(y_test,predicted_classes), precision_score(y_test, predicted_classes), recall_score(y_test, predicted_classes), f1_score(y_test, predicted_classes)

print('\n \n Scores')
scores = pd.DataFrame(data=[test_scores])
scores.columns = ['accuracy', 'precision', 'recall', 'f1']
scores = scores.T
scores.columns = ['scores']
display(scores)

Confusion Matrix


,0,1
0,2,653
1,0,0



 
 Scores


/Users/siddhipatel/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,scores
accuracy,0.003053
precision,0.000000
recall,0.000000
f1,0.000000
